In [1]:
from scipy import stats

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
%matplotlib inline

In [3]:
pre = pd.read_csv('DP_result.csv')

pre_df = pd.DataFrame(pre)
pre_df

,person_id,rnn,y_pred,y_real,gender,age,test_age,EGD,bmi,alcohol_habit,smoking status,smoking_duration,amount of smoking,income,family_cancer,current_status,exercise,cancer,date
0,41708664,0.974922,1,1,0,86.0,72.0,1.0,NaN,1.0,3.0,5,1,4.0,1,0,1.0,1,2006
1,41708664,0.974922,1,1,0,86.0,72.0,1.0,NaN,1.0,3.0,5,1,4.0,1,0,1.0,1,2006
2,41708664,0.974922,1,1,0,86.0,73.0,NaN,NaN,1.0,3.0,5,1,NaN,1,0,1.0,1,2007
3,41708664,0.974922,1,1,0,86.0,73.0,NaN,NaN,1.0,3.0,5,1,NaN,1,0,1.0,1,2007
4,41187947,0.974860,1,1,0,87.0,69.0,1.0,23.73,4.0,3.0,5,2,10.0,1,0,1.0,1,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12555,49785458,0.003307,0,0,0,56.0,42.0,NaN,28.09,1.0,3.0,4,3,9.0,1,0,1.0,0,2006
12556,49785458,0.003307,0,0,0,56.0,43.0,NaN,27.76,1.0,3.0,5,3,9.0,1,0,1.0,0,2007
12557,49785458,0.003307,0,0,0,56.0,44.0,NaN,27.76,3.0,3.0,4,3,9.0,0,0,1.0,0,2008
12558,49785458,0.003307,0,0,0,56.0,47.0,NaN,28.41,1.0,3.0,4,3,9.0,1,0,1.0,0,2011


In [4]:
pre_df1 = pre_df[(pre_df['y_real']==1)]
pre_df2 = pre_df[(pre_df['y_real']==0)]

In [5]:
persons1=list(set(pre_df1['person_id'].tolist()))
persons1.sort()
persons2=list(set(pre_df2['person_id'].tolist()))
persons2.sort()

In [6]:
print(len(persons1))
print(len(persons2))

187
5650


In [7]:
cs = pd.read_csv('Initial Care History.csv') # Initial Care History table
cs = pd.DataFrame(cs)

cancer = cs['current_status']==1
no_cancer= cs['current_status']==0
cs_c = cs[cancer]
cs_n = cs[no_cancer]

### Convert 2D array

In [8]:
# cancer of the stomach
new1 = pd.DataFrame(columns=['person_id','rnn','y_pred','y_real','gender','age','test_age','EGD','bmi','alcohol_habit','smoking status','smoking_duration','amount of smoking','income','family_cancer','current_status','exercise','cancer','date','observation_date'])

for p in persons1:
    ap = pre_df1['person_id']==p
    ap_df = pre_df1[ap]
    ap_leng = len(ap_df)
    #print(p)
    if ap_leng == 1:
        new1 = new1.append(ap_df)
    elif ap_leng >= 2:
        lengths =[]
        #print(p)
        cs_p = (cs_c[cs_c['person_id']==p]).iloc[0,3] # Gastric Cancer Date
        cs_p = pd.to_datetime(cs_p)
        
        for i in range(ap_leng):
            ap_p = (ap_df.iloc[i,18]) # Date of initial care recorded on the patient
            ap_p = pd.to_datetime(ap_p)
            if cs_p == ap_p:
                ap_pp = (ap_df.iloc[i,19]) # Health check up date recorded for the patient
                ap_pp = pd.to_datetime(ap_pp)
                # Compare with date of initial gastric cancer treatment, then compare date of health check up and difference
                lengths.append(abs((ap_pp-cs_p).days)) 
            else:
                lengths.append(99999) # Replace NULL value if initial care date is different
    
                
        min_p = lengths.index(min(lengths)) # Find and enter health check up records with the smallest date difference
        new1 = new1.append(ap_df.iloc[min_p,:])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


### Replace Null value & Convert to categorical variables

In [10]:
new1 = new1.replace({'EGD':np.nan},{'EGD':0})
new1 = new1.replace({'bmi':np.nan},{'bmi':round((new1['bmi']).mean(),2)})
new1 = new1.replace({'income':np.nan},{'income':(new1['income']).median()})
new1 = new1.replace({'exercise':np.nan},{'exercise':0})

In [11]:
df = pd.DataFrame(columns=['person_id', 'rnn', 'y_pred', 'y_real', 'gender', 'age', 'test_age',
       'EGD', 'bmi', 'alcohol_habit', 'smoking status', 'smoking_duration',
       'amount of smoking', 'income', 'family_cancer', 'current_status',
       'exercise', 'cancer', 'date', 'observation_date'])
for i in range(len(new1)):
    row = new1.iloc[i,:]
    
    row[5] = (row[5]//10)*10
    row[6] = (row[6]//10)*10
    
    if row[7]>0:
        row[7] = 1
    if row[8]<18.0:
        row[8] = 1
    elif ((row[8]>=18.0) & (row[8]<23.0)):
        row[8] = 2
    elif ((row[8]>=23.0) & (row[8]<25.0)):
        row[8] = 3
    elif row[8]>=25.0:
        row[8] = 4
    df.loc[i]=row

In [13]:
df['income'] = df['income'].replace([1,2,3],1)
df['income'] = df['income'].replace([4,5,6,7],2)
df['income'] = df['income'].replace([8,9,10],3)

### Chi-Square test

In [14]:
li = ['gender','age','test_age','EGD','bmi','alcohol_habit','smoking status','smoking_duration','amount of smoking','income','family_cancer','current_status','exercise']

for a in li:
    cross_tab = pd.crosstab(index=df[a],columns=df["y_pred"])
    chi2, p, dof, expected = stats.chi2_contingency(cross_tab) 
    msg = "Feature : {}, chi2:{}, p-value:{}, df:{}" 
    print(msg.format(a,chi2, p, dof))

Feature : gender, chi2:12.996551782806025, p-value:0.00031206512247137727, df:1
Feature : age, chi2:53.05172506926967, p-value:0.11797304563121103, df:42
Feature : test_age, chi2:36.70968819844077, p-value:0.3443245734875731, df:34
Feature : EGD, chi2:2.0452078038211585, p-value:0.5630768712156882, df:3
Feature : bmi, chi2:122.94483489666743, p-value:0.48444300737944723, df:123
Feature : alcohol_habit, chi2:13.128168112592446, p-value:0.06904649554475265, df:7
Feature : smoking status, chi2:18.50106382978723, p-value:9.606054243640363e-05, df:2
Feature : smoking_duration, chi2:20.92367154234097, p-value:0.0003278963901219573, df:4
Feature : amount of smoking, chi2:2.1212436793523155, p-value:0.5476269923891071, df:3
Feature : income, chi2:0.4009052407942655, p-value:0.8183602626914569, df:2
Feature : family_cancer, chi2:0.9881667384560868, p-value:0.3201908527696861, df:1
Feature : current_status, chi2:0.0, p-value:1.0, df:0
Feature : exercise, chi2:0.0, p-value:1.0, df:0
